In [23]:
import pyspark
from pyspark.sql import SparkSession
print("Hello")

Hello


In [27]:
#Minio Configuration
s3_host = "minio"
s3_url = f"http://{s3_host}:9000"
s3_key = "minio"
s3_secret = "SU2orange!"
s3_bucket = "labd"
print(s3_url)

http://minio:9000


In [29]:
#Spark init
spark = SparkSession.builder \
    .master("local") \
    .appName('jupyter-psyspark') \
    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.1.2") \
    .config("spark.hadoop.fs.s3a.endpoint", s3_url) \
    .config("spark.hadoop.fs.s3a.access.key", s3_key) \
    .config("spark.hadoop.fs.s3a.secret.key", s3_secret) \
    .config("spark.hadoop.fs.s3a.fast.upload", True) \
    .config("spark.hadoop.fs.s3a.path.style.access", True) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()
sc = spark.sparkContext
print(s3_bucket)
sc.setLogLevel("ERROR")

labd


In [30]:
s3_bucket = "labd"
print("Hello")
log_in = f"s3a://{s3_bucket}/logs/*.log"
print(log_in)
logs1 = spark.read.text(log_in)
logs1.show()

Hello
s3a://labd/logs/*.log
+--------------------+
|               value|
+--------------------+
|#Software: Micros...|
|       #Version: 1.0|
|#Date: 2016-02-11...|
|#Fields: date tim...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
+--------------------+
only showing top 20 rows



In [31]:
logs2 = logs1.filter(~logs1.value.startswith("#"))
logs2.show()

+--------------------+
|               value|
+--------------------+
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
+--------------------+
only showing top 20 rows



In [32]:
logs_out = f"s3a://{s3_bucket}/logs/logs_no_header"
logs2.write.text(logs_out)
print(logs_out)

s3a://labd/logs/logs_no_header


In [33]:
logs3 = spark.read.option("sep"," ").option("inferSchema", True).csv(logs_out)
logs3.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)



In [34]:
logs4 = spark.read.option("sep"," ").option("inferSchema", True).csv(logs_out) \
.toDF("date", "time", "serverip", "method", "uri", "querystring", "port", "username", "clientip", "useragent", "referrer", "statuscode", _, _, _)\
.select("date","time", "serverip", "method", "uri", "querystring", "port", "username", "clientip", "useragent", "referrer", "statuscode")
logs4.show()

+----------+--------+--------------+------+--------------------+-----------+----+--------+-----------+--------------------+--------------------+----------+
|      date|    time|      serverip|method|                 uri|querystring|port|username|   clientip|           useragent|            referrer|statuscode|
+----------+--------+--------------+------+--------------------+-----------+----+--------+-----------+--------------------+--------------------+----------+
|2016-02-11|17:16:13|128.230.247.37|   GET|                   /|          -|  80|       -|215.82.23.2|Mozilla/5.0+(Wind...|                   -|       200|
|2016-02-11|17:16:13|128.230.247.37|   GET|/Content/jquery-u...|          -|  80|       -|215.82.23.2|Mozilla/5.0+(Wind...|http://group0.ist...|       200|
|2016-02-11|17:16:13|128.230.247.37|   GET|/Plugins/Widgets....|          -|  80|       -|215.82.23.2|Mozilla/5.0+(Wind...|http://group0.ist...|       200|
|2016-02-11|17:16:13|128.230.247.37|   GET|/Plugins/Widgets....|

In [37]:
iplookup_in = f"s3a://{s3_bucket}/iplookup/iplookup.json"
ips1 = spark.read.option("multiline", True).json(iplookup_in)
ips1.printSchema()
ips1.show(10)

root
 |-- geography: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- state: string (nullable = true)
 |-- ip: string (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- lat: double (nullable = true)
 |    |-- lng: double (nullable = true)

+--------------------+---------------+--------------------+
|           geography|             ip|            location|
+--------------------+---------------+--------------------+
|   {Dulles, USA, VA}|  172.189.252.8|{38.955855, -77.4...|
| {Columbus, USA, OH}|    215.82.23.2|{39.961176, -82.9...|
|{Cleveland, USA, OH}|    98.29.25.44|{41.49932, -81.69...|
| {Freeport, USA, NY}|  68.199.40.156|{40.657602, -73.5...|
|{Salt Lake City, ...|155.100.169.152|{40.760779, -111....|
|   {Dallas, USA, TX}|   38.68.15.223|{32.776664, -96.7...|
|    {Tampa, USA, FL}|   70.209.14.54|{27.950575, -82.4...|
|{Arlington, USA, VA}|   74.111.6.173|{38.87997, -77.10...|
| {Syracuse,

In [40]:
ips2 = ips1.select("ip", "geography.city", "geography.state", "geography.country", ips1.location.lng, ips1.location.lng.alias("lng"))
ips2.show()

+---------------+--------------+-----+-------+------------+-----------+
|             ip|          city|state|country|location.lng|        lng|
+---------------+--------------+-----+-------+------------+-----------+
|  172.189.252.8|        Dulles|   VA|    USA|  -77.447819| -77.447819|
|    215.82.23.2|      Columbus|   OH|    USA|  -82.998794| -82.998794|
|    98.29.25.44|     Cleveland|   OH|    USA|  -81.694361| -81.694361|
|  68.199.40.156|      Freeport|   NY|    USA|  -73.583184| -73.583184|
|155.100.169.152|Salt Lake City|   UT|    USA| -111.891047|-111.891047|
|   38.68.15.223|        Dallas|   TX|    USA|  -96.796988| -96.796988|
|   70.209.14.54|         Tampa|   FL|    USA|  -82.457178| -82.457178|
|   74.111.6.173|     Arlington|   VA|    USA|   -77.10677|  -77.10677|
|128.230.122.180|      Syracuse|   NY|    USA|  -76.147424| -76.147424|
|128.122.140.238|      New York|   NY|    USA|  -74.005941| -74.005941|
| 56.216.127.219|       Raleigh|   NC|    USA|  -78.638179| -78.

In [43]:
comb1 = ips2.join(logs4, on = ips2.ip == logs4.clientip, how = "inner")
comb1.printSchema()
comb1.show(75)

root
 |-- ip: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- location.lng: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- serverip: string (nullable = true)
 |-- method: string (nullable = true)
 |-- uri: string (nullable = true)
 |-- querystring: string (nullable = true)
 |-- port: integer (nullable = true)
 |-- username: string (nullable = true)
 |-- clientip: string (nullable = true)
 |-- useragent: string (nullable = true)
 |-- referrer: string (nullable = true)
 |-- statuscode: integer (nullable = true)

+---------------+--------+-----+-------+------------+----------+----------+--------+--------------+------+--------------------+-----------+----+--------+---------------+--------------------+--------------------+----------+
|             ip|    city|state|country|location.lng|       lng|      date|  

In [44]:
cleaned_logs_out = f"s3a://{s3_bucket}/iplookup/cleaned-logs"
comb1.write.mode("Overwrite").parquet(cleaned_logs_out)